# Cape Basin Super Resolution

A simple notebook to test assorted SR models operating on the Cape Basin dataset

## Preparation

In [1]:
%matplotlib ipympl
import torch
import xarray as xa
import hydra, os
from fmod.base.util.config import fmconfig, cfg, ConfigContext
from fmod.model.sres.manager import ResultsAccumulator
from fmod.base.io.loader import TSet
from fmod.base.util.ops import fmbdir, fmtp
from fmod.base.source.loader import srRes

In [2]:
hydra.initialize(version_base=None, config_path="../config")
device = ConfigContext.set_device()
def pkey( model: str, tset: TSet, ltype: str ): return '-'.join([model,tset.value,ltype])

task="sres"
models = [ 'dbpn', 'edsr', 'srdn', 'unet', 'vdsr', 'mscnn' ]
dataset="LLC4320-v1"
scenario="s4.1"
fmconfig( task, models[0], dataset, scenario )

load_state  = "current"
save_state  = True 

### Results data
Read results summary from previous runs:

In [6]:
accum_results = ResultsAccumulator(task,dataset,scenario)
results = accum_results.read( fmbdir('processed') )

plots_data = {}
for model in models:
    for tset in [ TSet.Validation.value, TSet.Test.value ]:
        for ltype in [ 'model_loss', 'interp_loss' ]:
            plot_data = plots_data.setdefault( pkey(model,tset,ltype), {} )
            for result in results:
                if result['model'] == model and result['tset'] == tset:
                    plot_data[ result['epoch'] ] = result[ltype]
                
for model in models:
    print( f"{model}:" )
    for tset in [ TSet.Validation, TSet.Test ]:
        print( f"  * {tset.value}:" )
        for ltype in [ 'model_loss', 'interp_loss' ]:
            print( f"    ** {ltype}:" )
            pdata = plots_data[ pkey(model,tset,ltype) ]
            print( f"       {pdata}")
                
 